In [1]:
import numpy as np

from scipy.optimize import minimize, LinearConstraint

# part1

In [2]:
day15_inputs = "inputs/day15.txt"
with open(day15_inputs, 'r') as file:
    day15_data = file.readlines()

In [3]:
ingredients = {}
for ingredient in day15_data:
    ingredient_split = ingredient.split(" ")
    ingredient_name = ingredient_split[0].strip(":")

    ingredients.setdefault(ingredient_name, {})["capacity"] = int(ingredient_split[2].strip(","))
    ingredients.setdefault(ingredient_name, {})["durability"] = int(ingredient_split[4].strip(","))
    ingredients.setdefault(ingredient_name, {})["flavor"] = int(ingredient_split[6].strip(","))
    ingredients.setdefault(ingredient_name, {})["texture"] = int(ingredient_split[8].strip(","))
    ingredients.setdefault(ingredient_name, {})["calories"] = int(ingredient_split[10].strip(""))

In [4]:
ingredient_names = list(ingredients.keys())
property_names = ["capacity", "durability", "flavor", "texture"]

def total_score(vars):
    amounts = dict(zip(ingredient_names, vars))
    prop_sums = []
    for prop in property_names:
        s = sum(amounts[ing] * ingredients[ing][prop] for ing in ingredient_names)
        prop_sums.append(max(0, s))
    score = np.prod(prop_sums)
    return -score  # negative for maximization


In [5]:
# This seems like an optimisation problem where I have a multiparameter problem to maximise and variables to solve for

SUM = 100
x0 = np.full(len(ingredient_names), SUM / len(ingredient_names))
bounds = [(0, None)] * len(ingredient_names)

res = minimize(total_score, x0, bounds=bounds)
eq_constraint = {'type': 'eq', 'fun': lambda vars: vars[0] + vars[1] - SUM}

A = np.ones((1, len(ingredient_names)))
linear_cons = LinearConstraint(A, [SUM], [SUM])

res = minimize(total_score, x0, bounds=bounds, constraints=[linear_cons], method='trust-constr')
optimal_vars, optimal_total_score  = res.x, -res.fun
print(optimal_vars, optimal_total_score)

[24.16625836 29.26650332 30.4488173  16.11842102] 18982143.327538073


C:\Users\jacky\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scipy\optimize\_differentiable_functions.py:376: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(self.x - self.x_prev, self.g - self.g_prev)


In [6]:
# The result is close in the example testcase, however, the teaspoons have to be integer values
# Therefore, we assume that we are close, and we do a search of int(x, y, z, a)+-2, 
# and check their sum is 100, and calculate the total score for those combinations

optimal_vars_int = np.round(optimal_vars).astype(int)
total_score_calc = total_score(optimal_vars_int)
print(optimal_vars_int)

x_vals = np.arange(optimal_vars_int[0]-2, optimal_vars_int[0]+2, 1)
y_vals = np.arange(optimal_vars_int[1]-2, optimal_vars_int[1]+2, 1)
z_vals = np.arange(optimal_vars_int[2]-2, optimal_vars_int[2]+2, 1)
a_vals = np.arange(optimal_vars_int[3]-2, optimal_vars_int[3]+2, 1)

scores = []
for i in range(len(x_vals)):
    for j in range(len(y_vals)):
        for k in range(len(z_vals)):
            for l in range(len(a_vals)):
                if (x_vals[i] + y_vals[j] + z_vals[k] + a_vals[l] == SUM):
                    curr_total_score = total_score((x_vals[i], y_vals[j], z_vals[k], a_vals[l]))
                    scores.append(-int(curr_total_score))

print(scores)
print(max(scores))

[24 29 30 16]
[18693948, 18889992, 18853952, 18879744, 18908760, 18937116, 18965440, 18820836, 18957312, 18895500, 18744540, 18849600, 18878400, 18805060, 18939200, 18878400, 18604800, 18857600, 18900000, 18740400]
18965440


# part2

In [7]:
# I need to include a calorie constraint for the optimisation 
CALORIE_LIMIT = 500

In [8]:
def calorie_constraint(vars):
    amounts = dict(zip(ingredient_names, vars))
    total_cal = sum(amounts[ing] * ingredients[ing]["calories"] for ing in ingredient_names)
    return CALORIE_LIMIT - total_cal

In [9]:
x0 = np.full(len(ingredient_names), SUM / len(ingredient_names))
bounds = [(0, None)] * len(ingredient_names)

res = minimize(total_score, x0, bounds=bounds)
cal_constraint = {'type': 'ineq', 'fun': calorie_constraint}

A = np.ones((1, len(ingredient_names)))
linear_cons = LinearConstraint(A, [SUM], [SUM])

# include both the linear constraint and the calorie constraint
res = minimize(total_score, x0, bounds=bounds, constraints=[linear_cons, cal_constraint], method='trust-constr')
optimal_vars, optimal_total_score  = res.x, -res.fun
print(optimal_vars, optimal_total_score)

C:\Users\jacky\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\scipy\optimize\_differentiable_functions.py:728: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(delta_x, delta_g)


[22.15283183 22.87858866 30.24771041 24.7208691 ] 15960718.655094495


In [10]:
optimal_vars_int = np.round(optimal_vars).astype(int)
total_score_calc = total_score(optimal_vars_int)
print(optimal_vars_int)

x_vals = np.arange(optimal_vars_int[0]-2, optimal_vars_int[0]+2, 1)
y_vals = np.arange(optimal_vars_int[1]-2, optimal_vars_int[1]+2, 1)
z_vals = np.arange(optimal_vars_int[2]-2, optimal_vars_int[2]+2, 1)
a_vals = np.arange(optimal_vars_int[3]-2, optimal_vars_int[3]+2, 1)

scores = []
for i in range(len(x_vals)):
    for j in range(len(y_vals)):
        for k in range(len(z_vals)):
            for l in range(len(a_vals)):
                # Only consider values that meet both the sum and calorie constraints, i.e., SUM=100 and CALORIE_LIMIT=500
                if (
                    x_vals[i] + y_vals[j] + z_vals[k] + a_vals[l] == SUM
                    and sum(val * ingredients[ing]["calories"] for ing, val in zip(ingredient_names, [x_vals[i], y_vals[j], z_vals[k], a_vals[l]])) == CALORIE_LIMIT
                ):
                    curr_total_score = total_score((x_vals[i], y_vals[j], z_vals[k], a_vals[l]))
                    scores.append(-int(curr_total_score))

print(scores)
print(max(scores))

[22 23 30 25]
[15862900, 15628800]
15862900
